In [450]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing, Holt
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.holtwinters import Holt
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import warnings
import datetime
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
import openpyxl
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
import os
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA

In [452]:
df1 = pd.read_csv('file:///Users/marycarolinekreps/Downloads/rptSales.csv')
df1.head()

,InvoiceNumber,LineNumber,InvoiceDate,ReqDate,SalesOrderType,TransactionType,RevenueAccount,RevenueAccountDesc,RevenueAccountType,RevenueAccountTerm,...,PricePerLB,CostPerLB,QtyInGallons,MaterialMargin,MaterialMarginPercent,TAMId,IsInFull,IsOnTime,InvMonth,InvYear
0,85757,16,7/1/16,NaN,NaN,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,0.000000,0.000000,NaN,0.0000,NaN,FlooringPALLET|7,NaN,0,7,2016
1,90912,28,12/16/16,NaN,NaN,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,0.160000,0.000000,NaN,8.0000,1.000000,FlooringPALLET|7,NaN,0,12,2016
2,92476,16,2/9/17,2/13/17,Normal,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,1.092051,0.572177,NaN,20.2751,0.476053,Flooring2033-1|7,1.0,0,2,2017
3,92476,28,2/9/17,2/13/17,Normal,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,0.340000,0.348738,NaN,-0.3670,-0.025700,Flooring960-4-TA|7,1.0,0,2,2017
4,102400,16,11/30/17,11/21/17,Normal,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,2.319143,1.138360,NaN,41.3274,0.509146,Flooring2091-1|7,1.0,1,11,2017


In [454]:
item = pd.read_csv('file:///Users/marycarolinekreps/Desktop/ItemRpt.csv')

In [456]:
df1_item = df1.merge(item[['ItemId','ItemDescription','ItemClass','LastCostPaid','AverageCost','StandardCost','ClassDescription','CommCodeDescription','ProdLineDescription','MonthsUntilExpired','BusinessUnit','StockUoM']],on='ItemId', how='left')
df1_item['InvoiceDate'] = pd.to_datetime(df1_item['InvoiceDate'], format='%m/%d/%y')


In [458]:
df1_item.head()

,InvoiceNumber,LineNumber,InvoiceDate,ReqDate,SalesOrderType,TransactionType,RevenueAccount,RevenueAccountDesc,RevenueAccountType,RevenueAccountTerm,...,ItemClass,LastCostPaid,AverageCost,StandardCost,ClassDescription,CommCodeDescription,ProdLineDescription,MonthsUntilExpired,BusinessUnit,StockUoM
0,85757,16,2016-07-01,NaN,NaN,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,5199,0.00000,0.00000,0.00,FG Pallet Sales ...,No Comm Code Assigned,Pallet Sales ...,0.0,Flooring,EA
1,90912,28,2016-12-16,NaN,NaN,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,5199,0.00000,0.00000,0.00,FG Pallet Sales ...,No Comm Code Assigned,Pallet Sales ...,0.0,Flooring,EA
2,92476,16,2017-02-09,2/13/17,Normal,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,5100,20.71780,22.81510,14.42,FG Latex Adhesive ...,Steadfast - Clear Thin Spread VCT Adhesive ...,Latex Distributor Sales ...,365.0,Flooring,CTN-1
3,92476,28,2017-02-09,2/13/17,Normal,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,5100,15.30572,15.98697,0.00,FG Latex Adhesive ...,900 - Contract Grade Flooring Adhesive ...,Latex Distributor Sales ...,365.0,Flooring,EA-4
4,102400,16,2017-11-30,11/21/17,Normal,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,5142,40.66792,2.54175,0.00,FG Meta-Tec Adhesives ...,Pinnacle - Hard-Set Transitional PSA ...,Meta-Tec Adhesives ...,365.0,Flooring,CTN-1


In [460]:

def add_time_features(df1_item, lags=5, rolling_window=3, include_seasonality=False):
    """
    Adds lag, rolling window, and optional seasonal features to the df1_item DataFrame.

    Parameters:
    - df1_item (pd.DataFrame): DataFrame with 'RevenueAmount' and datetime index.
    - lags (int): Number of lagged features to create.
    - rolling_window (int): Window size for rolling statistics.
    - include_seasonality (bool): Whether to add seasonal dummy variables.

    Returns:
    - pd.DataFrame: DataFrame with added features.
    """
    # Validate input DataFrame
    required_columns = ['RevenueAmount']
    for col in required_columns:
        if col not in df1_item.columns:
            raise ValueError(f"Missing required column: '{col}'")

    # Ensure the index is a datetime index
    if not isinstance(df1_item.index, pd.DatetimeIndex):
        raise TypeError("DataFrame index must be a pandas DatetimeIndex.")

    # Sort the DataFrame by index to ensure chronological order
    df1_item = df1_item.sort_index()

    # Add lag features
    for lag in range(1, lags + 1):
        lag_col = f'lag_{lag}'
        df1_item[lag_col] = df1_item['RevenueAmount'].shift(lag)
        #print(f"Added lag feature: {lag_col}")

    # Add rolling window features
    df1_item[f'rolling_mean_{rolling_window}'] = df1_item['RevenueAmount'].rolling(window=rolling_window).mean()
    df1_item[f'rolling_std_{rolling_window}'] = df1_item['RevenueAmount'].rolling(window=rolling_window).std()
    #print(f"Added rolling_mean_{rolling_window} and rolling_std_{rolling_window}")

    # Add date-based features
    df1_item['day_of_week'] = df1_item.index.dayofweek       # Monday=0, Sunday=6
    df1_item['month'] = df1_item.index.month                 # 1 to 12
    df1_item['quarter'] = df1_item.index.quarter             # 1 to 4
    #print("Added day_of_week, month, and quarter features")

    # Optionally add seasonal dummy variables
    if include_seasonality:
        # Example: Add monthly dummy variables
        month_dummies = pd.get_dummies(df1_item.index.month, prefix='month', drop_first=True)
        df1_item = pd.concat([df1_item, month_dummies], axis=1)
        #print("Added monthly seasonal dummy variables")

    # Drop rows with NaN values resulting from lagging and rolling
    initial_shape = df1_item.shape
    df1_item.dropna(inplace=True)
    final_shape = df1_item.shape
    #print(f"Dropped rows with NaN values: {initial_shape[0] - final_shape[0]} rows removed")

    return df1_item


In [462]:
# ============================
# Forecasting Models
# ============================

def create_lagged_features(data, lags=5):
    """Create lag features for time series data."""
    df = pd.DataFrame(data, columns=['demand'])
    for lag in range(1, lags + 1):
        df[f'lag_{lag}'] = df['demand'].shift(lag)
    df.dropna(inplace=True)
    if df.empty:
        raise ValueError("Not enough data after creating lagged features.")
    return df

def forecast_random_forest(train, steps=30, lags=5):
    """Forecast using Random Forest model."""
    # Create lagged features
    train_lagged = create_lagged_features(train, lags)
    
    # Split into features (X) and target (y)
    X_train = train_lagged.drop(columns=['demand'])
    y_train = train_lagged['demand']
    
    # Check if there's enough data
    if X_train.shape[0] == 0:
        raise ValueError("Training data has 0 samples after processing.")
    
    # Initialize the Random Forest model
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Forecast future values
    forecast = []
    last_values = train[-lags:].tolist()  # Get the last `lags` values to start forecasting
    
    for step in range(steps):
        # Create the feature vector for the current step forecast
        X_step = np.array(last_values[-lags:]).reshape(1, -1)
        next_value = model.predict(X_step)[0]
        forecast.append(next_value)
        
        # Update the last_values list with the predicted value
        last_values.append(next_value)
    
    return np.array(forecast)

def forecast_xgboost_model(train, steps=30, lags=5):
    """Forecast using XGBoost model."""
    # Create lagged features
    train_lagged = create_lagged_features(train, lags)
    
    # Split into features (X) and target (y)
    X_train = train_lagged.drop(columns=['demand'])
    y_train = train_lagged['demand']
    
    # Check if there's enough data
    if X_train.shape[0] == 0:
        raise ValueError("Training data has 0 samples after processing.")
    
    # Initialize the XGBoost Regressor
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=42)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Forecast future values
    forecast = []
    last_values = train[-lags:].tolist()  # Get the last `lags` values to start forecasting
    
    for step in range(steps):
        # Create the feature vector for the current step forecast
        X_step = np.array(last_values[-lags:]).reshape(1, -1)
        next_value = model.predict(X_step)[0]
        forecast.append(next_value)
        
        # Update the last_values list with the predicted value
        last_values.append(next_value)
    
    return np.array(forecast)
'''
def forecast_moving_average(data, window=3, steps=18):
    """
    Moving average forecast for a dataset with at least 5 data points.
    Returns forecast values based on moving average if data suffices.
    """
    if len(data) < 5:
        # Not enough data to generate a forecast; skip this forecast
        return None
    else:
        # Use the last available moving average as the forecast
        forecast_value = data['RevenueAmount'].rolling(window=window).mean().iloc[-1]
        forecast = [forecast_value] * steps
        return forecast
'''

def forecast_exponential_smoothing(train, steps=30):
    """Forecast using Exponential Smoothing."""
    model = ExponentialSmoothing(train, seasonal=None)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps)
    return np.array(forecast)


        
def forecast_arima_model(train, steps=30, order=(5,1,0)):
    """Forecast using ARIMA model."""
    model = ARIMA(train, order=order)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps)
    return np.array(forecast)

def forecast_linear_regression(train, steps=30, lags=5):
    """Forecast using Linear Regression."""
    # Create lagged features
    train_lagged = create_lagged_features(train, lags)
    
    # Split into features (X) and target (y)
    X_train = train_lagged.drop(columns=['demand'])
    y_train = train_lagged['demand']
    
    # Initialize the Linear Regression model
    model = LinearRegression()
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Forecast future values
    forecast = []
    last_values = train[-lags:].tolist()
    
    for step in range(steps):
        X_step = np.array(last_values[-lags:]).reshape(1, -1)
        next_value = model.predict(X_step)[0]
        forecast.append(next_value)
        last_values.append(next_value)
    
    return np.array(forecast)

def forecast_xgboost_model(train, steps=30, lags=5):
    """Forecast using XGBoost model."""
    # Create lagged features
    train_lagged = create_lagged_features(train, lags)
    
    # Split into features (X) and target (y)
    X_train = train_lagged.drop(columns=['demand'])
    y_train = train_lagged['demand']
    
    # Check if there's enough data
    if X_train.shape[0] == 0:
        raise ValueError("Training data has 0 samples after processing.")
    
    # Initialize the XGBoost Regressor
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, random_state=42)
    
    # Train the model
    model.fit(X_train, y_train)
    
    # Forecast future values
    forecast = []
    last_values = train[-lags:].tolist()  # Get the last `lags` values to start forecasting
    
    for step in range(steps):
        # Create the feature vector for the current step forecast
        X_step = np.array(last_values[-lags:]).reshape(1, -1)
        next_value = model.predict(X_step)[0]
        forecast.append(next_value)
        
        # Update the last_values list with the predicted value
        last_values.append(next_value)
    
    return np.array(forecast)


In [464]:
# ============================
# Evaluation Metrics
# ============================

def evaluate_forecasts(y_true, y_pred):
    # Check for shape alignment
    if len(y_true) != len(y_pred):
        print(f"Shape mismatch: y_true ({len(y_true)}) vs y_pred ({len(y_pred)})")
        return np.nan, np.nan

    try:
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    except Exception as e:
        print(f"Error in evaluate_forecasts: {e}")
        return np.nan, np.nan  # Return NaN if an error occurs

    return mae, rmse

In [466]:
def aggregate_data(df1_item, forecast_by):
    """
    Aggregates revenue data by the selected dimension.

    Parameters:
    - df1_item (DataFrame): The input DataFrame containing sales data.
    - forecast_by (str): The column to group by ('ItemId' or 'ProdLineDescription').

    Returns:
    - DataFrame: Aggregated DataFrame.
    """
    if forecast_by not in ['ItemId', 'ProdLineDescription']:
        raise ValueError("forecast_by must be either 'ItemId' or 'ProdLineDescription'")
    
    # Group by the selected dimension and aggregate revenue
    aggregated = df1_item.groupby([forecast_by, 'InvoiceDate'])['RevenueAmount'].sum().reset_index()
    
    return aggregated

In [468]:
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
import matplotlib.pyplot as plt

def forecasting_pipeline(group_df, steps=30):
    """
    Forecasting pipeline for a single group, including multiple models.

    Parameters:
    - group_df (pd.DataFrame): DataFrame filtered for a specific ItemId or ProdLineDescription.
    - steps (int): Number of periods to forecast.

    Returns:
    - best_model (str): Name of the best-performing model based on MAE.
    - final_forecast (np.array): Forecasted values from the best model.
    - metrics (dict): Evaluation metrics for all models.
    """
    # Prepare data
    group_df = group_df.sort_values('InvoiceDate')
    group_df['InvoiceDate'] = pd.to_datetime(group_df['InvoiceDate'])
    group_df = group_df[~group_df.index.duplicated(keep='first')]
    group_df.set_index('InvoiceDate', inplace=True)
    inferred_freq = pd.infer_freq(group_df.index) or 'D'
    group_df = group_df.asfreq(inferred_freq)
    group_df.ffill(inplace=True)  # Fill missing values with forward fill
    group_df.dropna(inplace=True)

    try:
        group_df = add_time_features(group_df, lags=5, rolling_window=3)
    except Exception as e:
        print(f"Error in adding time features: {e}")
        return None, None, {}

    if len(group_df) < steps + 1:
        print("Not enough data to split into train and test.")
        return None, None, {}

    train = group_df.iloc[:-steps]
    test = group_df.iloc[-steps:]
    feature_cols = [col for col in group_df.columns if col != 'RevenueAmount']
    X_train, y_train = train[feature_cols], train['RevenueAmount']
    X_test, y_test = test[feature_cols], test['RevenueAmount']

    forecasts = {}
    metrics = {}

    # Moving Average Model
    print("Evaluating Moving Average model...")
    if len(train) >= 5:
        try:
            ma_forecast = forecast_moving_average(train, window=3, steps=steps)
            ma_mae, ma_rmse = evaluate_forecasts(y_test, ma_forecast)
            forecasts['MovingAverage'] = ma_forecast
            metrics['MovingAverage'] = {'MAE': ma_mae, 'RMSE': ma_rmse}
            print(f"Moving Average MAE: {ma_mae}, RMSE: {ma_rmse}")
        except Exception as e:
            print(f"Moving Average Error: {e}")

    # Exponential Smoothing Model
    try:
        print("Evaluating Exponential Smoothing model...")
        exp_model = ExponentialSmoothing(y_train, seasonal='additive', seasonal_periods=12)
        exp_fit = exp_model.fit()
        exp_forecast = exp_fit.forecast(steps=len(y_test))
        exp_mae, exp_rmse = evaluate_forecasts(y_test, exp_forecast)
        forecasts['ExponentialSmoothing'] = exp_forecast
        metrics['ExponentialSmoothing'] = {'MAE': exp_mae, 'RMSE': exp_rmse}
        print(f"Exponential Smoothing MAE: {exp_mae}, RMSE: {exp_rmse}")
    except Exception as e:
        print(f"Exponential Smoothing Error: {e}")

    # XGBoost Model
    try:
        print("Evaluating XGBoost model...")
        xgb_model = XGBRegressor(objective='reg:squarederror', seed=42)
        xgb_model.fit(X_train, y_train)
        xgb_forecast = xgb_model.predict(X_test)
        xgb_mae, xgb_rmse = evaluate_forecasts(y_test, xgb_forecast)
        forecasts['XGBoost'] = xgb_forecast
        metrics['XGBoost'] = {'MAE': xgb_mae, 'RMSE': xgb_rmse}
        print(f"XGBoost MAE: {xgb_mae}, RMSE: {xgb_rmse}")
    except Exception as e:
        print(f"XGBoost Error: {e}")

    # ARIMA Model
    try:
        print("Evaluating ARIMA model...")
        model_arima = ARIMA(train['RevenueAmount'], order=(5, 1, 0))
        model_arima_fit = model_arima.fit()
        arima_forecast = model_arima_fit.forecast(steps).values
        arima_mae, arima_rmse = evaluate_forecasts(y_test, arima_forecast)
        forecasts['ARIMA'] = arima_forecast
        metrics['ARIMA'] = {'MAE': arima_mae, 'RMSE': arima_rmse}
        print(f"ARIMA MAE: {arima_mae}, RMSE: {arima_rmse}")
    except Exception as e:
        print(f"ARIMA Error: {e}")

    # Linear Regression Model
    try:
        print("Evaluating Linear Regression model...")
        lr_model = LinearRegression()
        lr_model.fit(X_train, y_train)
        lr_forecast = lr_model.predict(X_test)
        lr_mae, lr_rmse = evaluate_forecasts(y_test, lr_forecast)
        forecasts['LinearRegression'] = lr_forecast
        metrics['LinearRegression'] = {'MAE': lr_mae, 'RMSE': lr_rmse}
        print(f"Linear Regression MAE: {lr_mae}, RMSE: {lr_rmse}")
    except Exception as e:
        print(f"Linear Regression Error: {e}")

    # Determine best model based on lowest MAE
    valid_metrics = {k: v for k, v in metrics.items() if not np.isnan(v['MAE'])}

    if not valid_metrics:
        print("No valid models were generated.")
        return None, None, metrics

    # Select the model with the lowest MAE
    best_model = min(valid_metrics, key=lambda x: valid_metrics[x]['MAE'])
    final_forecast = forecasts[best_model]
    print(f"Best model selected: {best_model}")

    return best_model, final_forecast, metrics


In [470]:
# ============================
# Running Forecasting for All Groups
# ============================

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

def run_forecasting(df1_item, forecast_by, steps=30, selected_model=None):
    """
    Runs forecasting for all groups based on the selected dimension.
    
    Parameters:
    - df1_item (DataFrame): The input dataframe.
    - forecast_by (str): The column name to forecast by ('ItemId' or 'ProdLineDescription').
    - steps (int): Number of periods to forecast.
    - selected_model (str or None): Specific model to run or None for Best Fit.
    
    Returns:
    - dict: Forecasts for each group.
    - dict: Selected models for each group.
    - dict: Metrics for each group.
    """
    aggregated_df = aggregate_data(df1_item, forecast_by)
    groups = aggregated_df[forecast_by].unique()
    
    all_forecasts = {}
    selected_models = {}
    all_metrics = {}
    
    for group in groups:
        group_df = aggregated_df[aggregated_df[forecast_by] == group]
        if len(group_df) < steps + 1:
            print(f"\nGroup '{group}' has insufficient data for forecasting. Skipping.")
            continue
        
        if selected_model == 'Best Fit' or selected_model is None:
            print(f"\nRunning Best Fit model for {forecast_by}: {group}")
            best_model, forecast, metrics = forecasting_pipeline(group_df, steps)
        else:
            print(f"\nRunning {selected_model} model for {forecast_by}: {group}")
            best_model, forecast, metrics = run_specific_model(group_df, steps, selected_model)
        
        if best_model is not None:
            all_forecasts[group] = forecast
            selected_models[group] = best_model
            all_metrics[group] = metrics
    
    return all_forecasts, selected_models, all_metrics

def run_specific_model(group_df, steps, model_name):
    """
    Runs a specific forecasting model for a single group.
    """
    # Sort by date
    group_df = group_df.sort_values('InvoiceDate')
    
    # Prepare data for training and testing
    if len(group_df) < steps + 1:
        print("Not enough data to split into train and test.")
        return None, None, {}
    
    group_df['date_num'] = (group_df['InvoiceDate'] - group_df['InvoiceDate'].min()).dt.days  # Convert date to numerical
    X = group_df[['date_num']]  # Features: date as numerical
    y = group_df['RevenueAmount']  # Target: revenue amount

    # Initialize dictionaries to store forecasts and metrics
    forecasts = {}
    metrics = {}
    
    # Select and run the chosen model
    if model_name == 'RandomForest': 
        try:
            # Train Random Forest model
            model = RandomForestRegressor(n_estimators=100)
            model.fit(X[:-steps], y[:-steps])  # Fit on training data

            # Prepare future dates for forecasting
            future_dates = pd.date_range(start=group_df['InvoiceDate'].max() + pd.Timedelta(days=1), 
                                          periods=steps)
            future_date_nums = (future_dates - group_df['InvoiceDate'].min()).days.values.reshape(-1, 1)  # Convert to numerical

            # Make predictions
            forecast = model.predict(future_date_nums)
            
            # Evaluate the forecast against the actual values (if available)
            test = y[-steps:]
            mae = mean_absolute_error(test, forecast) if len(test) > 0 else np.nan
            rmse = mean_squared_error(test, forecast, squared=False) if len(test) > 0 else np.nan

            forecasts['RandomForest'] = forecast
            metrics['RandomForest'] = {'MAE': mae, 'RMSE': rmse}
            best_model = 'RandomForest'

        except Exception as e:
            forecasts['RandomForest'] = np.array([np.nan] * steps)
            metrics['RandomForest'] = {'MAE': np.nan, 'RMSE': np.nan}
            print(f"Random Forest Error: {e}")
            best_model = None
            
    else:
        best_model = None
        forecast = np.array([np.nan] * steps)
        metrics = {}
    
    return best_model, forecast, metrics


In [472]:
# ============================
# Visualization Function
# ============================

import matplotlib.pyplot as plt
import pandas as pd

def plot_forecast(group_df, forecast_df, forecast_periods, group_name, forecast_col):
    """
    Plots the actual vs forecasted values for a group.

    Parameters:
    - group_df (DataFrame): Historical data for the group (before forecast).
    - forecast_df (DataFrame): DataFrame containing the forecast values.
    - forecast_periods (int): Number of periods that were forecasted.
    - group_name (str): The name of the group being plotted (e.g., product_id).
    - forecast_col (str): The column name that was forecasted (e.g., 'sales', 'revenue').
    """
    # Extract historical and forecast data
    actual_values = group_df[forecast_col]
    forecast_values = forecast_df['forecast'][-forecast_periods:]  # Get only forecasted values for plotting
    
    # Create a figure for plotting
    plt.figure(figsize=(12, 6))
    
    # Plot historical data
    plt.plot(group_df.index, actual_values, label='Historical', color='blue', marker='o', linestyle='-')
    
    # Plot forecasted data (continuing from the last historical date)
    forecast_dates = pd.date_range(start=group_df.index[-1], periods=forecast_periods + 1, freq='M')[1:]
    plt.plot(forecast_dates, forecast_values, label='Forecasted', color='orange', marker='o', linestyle='--')
    
    # Set plot title and labels
    plt.title(f'Actual vs Forecasted {forecast_col.capitalize()} for {group_name}')
    plt.xlabel('Date')
    plt.ylabel(f'{forecast_col.capitalize()}')
    
    # Add a legend
    plt.legend()
    
    # Add grid lines
    plt.grid(True)
    
    # Show plot with tight layout
    plt.tight_layout()
    plt.show()

# Example call for plotting (assuming you have a forecast result for 'group_1'):
# group_df: Historical data for the group
# forecast_results['group_1']: Contains both historical and forecast data for 'group_1'
# plot_forecast(group_df, forecast_results['group_1'], forecast_periods, 'group_1', forecast_column)


In [474]:
def filter_data(df1_item):
    """Function to filter data based on user inputs."""
    
    # Ask whether to filter by CustomerId
    filter_customer = input("Would you like to filter by 'CustomerId'? Type 'no' for no filtering or 'yes' to select a customer: ").strip().lower()
    
    if filter_customer == 'yes':
        # List available CustomerIds and allow the user to select one
        unique_customers = df1_item['CustomerId'].unique()
        print("\nAvailable Customers:")
        for idx, customer in enumerate(unique_customers, start=1):
            print(f"{idx}. {customer}")
        
        customer_choice = int(input("\nSelect the number corresponding to the CustomerId you want to filter by: ")) - 1
        selected_customer = unique_customers[customer_choice]
        df1_item = df1_item[df1_item['CustomerId'] == selected_customer]
        print(f"Filtering by CustomerId: {selected_customer}")
    else:
        print("No filtering by CustomerId.")

    # Ask whether to filter by CurrentSalesPersonId
    filter_salesperson = input("Would you like to filter by 'CurrentSalesPersonId'? Type 'no' for no filtering or 'yes' to select a salesperson: ").strip().lower()
    
    if filter_salesperson == 'yes':
        # List available SalesPersonIds and allow the user to select one
        unique_salespersons = df1_item['CurrentSalesPersonId'].unique()
        print("\nAvailable SalesPersons:")
        for idx, salesperson in enumerate(unique_salespersons, start=1):
            print(f"{idx}. {salesperson}")
        
        salesperson_choice = int(input("\nSelect the number corresponding to the SalesPersonId you want to filter by: ")) - 1
        selected_salesperson = unique_salespersons[salesperson_choice]
        df1_item = df1_item[df1_item['CurrentSalesPersonId'] == selected_salesperson]
        print(f"Filtering by SalesPersonId: {selected_salesperson}")
    else:
        print("No filtering by CurrentSalesPersonId.")

    # Ask whether to filter by LocationId
    filter_location = input("Would you like to filter by 'LocationId'? Type 'no' for no filtering or 'yes' to select a location: ").strip().lower()
    
    if filter_location == 'yes':
        # List available LocationIds and allow the user to select one
        unique_locations = df1_item['LocationId'].unique()
        print("\nAvailable Locations:")
        for idx, location in enumerate(unique_locations, start=1):
            print(f"{idx}. {location}")
        
        location_choice = int(input("\nSelect the number corresponding to the LocationId you want to filter by: ")) - 1
        selected_location = unique_locations[location_choice]
        df1_item = df1_item[df1_item['LocationId'] == selected_location]
        print(f"Filtering by LocationId: {selected_location}")
    else:
        print("No filtering by LocationId.")
    
    return df1_item


In [476]:
df1_item.head()

,InvoiceNumber,LineNumber,InvoiceDate,ReqDate,SalesOrderType,TransactionType,RevenueAccount,RevenueAccountDesc,RevenueAccountType,RevenueAccountTerm,...,ItemClass,LastCostPaid,AverageCost,StandardCost,ClassDescription,CommCodeDescription,ProdLineDescription,MonthsUntilExpired,BusinessUnit,StockUoM
0,85757,16,2016-07-01,NaN,NaN,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,5199,0.00000,0.00000,0.00,FG Pallet Sales ...,No Comm Code Assigned,Pallet Sales ...,0.0,Flooring,EA
1,90912,28,2016-12-16,NaN,NaN,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,5199,0.00000,0.00000,0.00,FG Pallet Sales ...,No Comm Code Assigned,Pallet Sales ...,0.0,Flooring,EA
2,92476,16,2017-02-09,2/13/17,Normal,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,5100,20.71780,22.81510,14.42,FG Latex Adhesive ...,Steadfast - Clear Thin Spread VCT Adhesive ...,Latex Distributor Sales ...,365.0,Flooring,CTN-1
3,92476,28,2017-02-09,2/13/17,Normal,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,5100,15.30572,15.98697,0.00,FG Latex Adhesive ...,900 - Contract Grade Flooring Adhesive ...,Latex Distributor Sales ...,365.0,Flooring,EA-4
4,102400,16,2017-11-30,11/21/17,Normal,Invoiced/Voided Order,NaN,NaN,NaN,NaN,...,5142,40.66792,2.54175,0.00,FG Meta-Tec Adhesives ...,Pinnacle - Hard-Set Transitional PSA ...,Meta-Tec Adhesives ...,365.0,Flooring,CTN-1


In [478]:
print(df1_item.columns)

Index(['InvoiceNumber', 'LineNumber', 'InvoiceDate', 'ReqDate',
       'SalesOrderType', 'TransactionType', 'RevenueAccount',
       'RevenueAccountDesc', 'RevenueAccountType', 'RevenueAccountTerm',
       'RevenueAccountCategory', 'RevenueAccountTypeCode', 'COGSAccount',
       'CostAccountDesc', 'CostAccountType', 'CostAccountTerm',
       'CostAccountCategory', 'CostAccountTypeCode', 'EntityCode',
       'CustomerId', 'CustomerShipToKey', 'SalesPersonId',
       'CurrentSalesPersonId', 'DealerNumber', 'LocationId', 'ItemId',
       'DiscountPercent', 'DiscountAmount', 'glsale', 'glasst',
       'UnitOfMeasure', 'arstat', 'QuantityShipped', 'price', 'RevenueAmount',
       'UnitCost', 'CostAmount', 'PricePerGallon', 'CostPerGallon',
       'WeightPerUnit', 'PricePerLB', 'CostPerLB', 'QtyInGallons',
       'MaterialMargin', 'MaterialMarginPercent', 'TAMId', 'IsInFull',
       'IsOnTime', 'InvMonth', 'InvYear', 'ItemDescription', 'ItemClass',
       'LastCostPaid', 'AverageCost', 'Stan

In [480]:
def main(df1_item):
    """Main function to run the forecasting and save to Excel with metadata."""
    
    # Ask user if they want the full 18-month forecast or a single forecast
    forecast_option = input("Would you like to generate the 'full' 18-month forecast or a 'single' forecast? (Enter 'full' or 'single'): ").strip().lower()

    if forecast_option == 'full':
        try:
            unique_combinations = df1_item[['ItemId', 'LocationId']].drop_duplicates()
            wb = Workbook()
            ws = wb.active
            ws.title = "Forecast Data"
            
            headers = ["Part Number", "Location", "Type", "Model", "Month", "Value"]
            ws.append(headers)

            for _, row in unique_combinations.iterrows():
                item_id = row['ItemId']
                location_id = row['LocationId']
                data_subset = df1_item[(df1_item['ItemId'] == item_id) & (df1_item['LocationId'] == location_id)]
                
                if data_subset.empty:
                    print(f"No data available for Part Number {item_id} at Location {location_id}. Skipping.")
                    continue

                data_subset = data_subset.drop_duplicates(subset=['InvoiceDate'])
                for _, actual_row in data_subset.iterrows():
                    if pd.notnull(actual_row['InvoiceDate']):
                        month_value = actual_row['InvoiceDate'].strftime('%Y-%m')
                    else:
                        month_value = "Unknown Date"
                    
                    ws.append([item_id, location_id, "Actuals", "", month_value, actual_row['RevenueAmount']])

                steps = 18
                if len(data_subset) < 3:
                    forecast = forecast_moving_average(data_subset, steps=steps)
                    model_name = "Moving Average"
                else:
                    best_model, forecast, metrics = forecasting_pipeline(data_subset, steps=steps)
                    model_name = best_model if best_model else "Moving Average"
                    
                    if forecast is None:
                        forecast = forecast_moving_average(data_subset, steps=steps)
                        model_name = "Moving Average"
                        if forecast is None:
                            print(f"Insufficient data to forecast for Part Number {item_id} at Location {location_id}. Skipping.")
                            continue

                # Verify that forecast is not None before proceeding
                if forecast is not None:
                    for month, value in enumerate(forecast, start=1):
                        ws.append([item_id, location_id, "Forecast", model_name, f"Month_{month}", value])

            output_file = './part_numbers_18_month_forecast.xlsx'
            wb.save(output_file)
            print(f"Full 18-month forecast Excel spreadsheet created successfully: {output_file}")

        except Exception as e:
            print(f"An error occurred while creating the Excel spreadsheet for the full forecast: {e}")
        return
           
    elif forecast_option == 'single':
        # The existing single forecast implementation remains unchanged
        forecast_by = input("Would you like to forecast by 'ItemId' or 'ProdLineDescription'? ").strip()

        if forecast_by not in ['ItemId', 'ProdLineDescription']:
            print("Invalid input. Please choose 'ItemId' or 'ProdLineDescription'.")
            return

        df1_item['InvoiceDate'] = pd.to_datetime(df1_item['InvoiceDate'])

        # Filter options
        filter_by_customer = input("Would you like to filter by 'CustomerId'? (yes/no): ").strip().lower()
        if filter_by_customer == 'yes':
            customer_ids = df1_item['CustomerId'].unique()
            print("Available Customers:")
            for idx, customer in enumerate(customer_ids, start=1):
                print(f"{idx}. {customer}")
            customer_choice = int(input("Select the number corresponding to the Customer: ")) - 1
            if customer_choice < 0 or customer_choice >= len(customer_ids):
                print("Invalid selection.")
                return
            selected_customer = customer_ids[customer_choice]
            df1_item = df1_item[df1_item['CustomerId'] == selected_customer]
        else:
            selected_customer = 'None'

        filter_by_salesperson = input("Would you like to filter by 'CurrentSalesPersonId'? (yes/no): ").strip().lower()
        if filter_by_salesperson == 'yes':
            salesperson_ids = df1_item['CurrentSalesPersonId'].unique()
            print("Available Salespersons:")
            for idx, salesperson in enumerate(salesperson_ids, start=1):
                print(f"{idx}. {salesperson}")
            salesperson_choice = int(input("Select the number corresponding to the Salesperson: ")) - 1
            if salesperson_choice < 0 or salesperson_choice >= len(salesperson_ids):
                print("Invalid selection.")
                return
            selected_salesperson = salesperson_ids[salesperson_choice]
            df1_item = df1_item[df1_item['CurrentSalesPersonId'] == selected_salesperson]
        else:
            selected_salesperson = 'None'

        filter_by_location = input("Would you like to filter by 'LocationId'? (yes/no): ").strip().lower()
        if filter_by_location == 'yes':
            location_ids = df1_item['LocationId'].unique()
            print("Available Locations:")
            for idx, location in enumerate(location_ids, start=1):
                print(f"{idx}. {location}")
            location_choice = int(input("Select the number corresponding to the Location: ")) - 1
            if location_choice < 0 or location_choice >= len(location_ids):
                print("Invalid selection.")
                return
            selected_location = location_ids[location_choice]
            df1_item = df1_item[df1_item['LocationId'] == selected_location]
        else:
            selected_location = 'None'

        aggregated_data = aggregate_data(df1_item, forecast_by)
        unique_items = aggregated_data[forecast_by].unique()

        print(f"\nAvailable {forecast_by}s:")
        for idx, item in enumerate(unique_items, start=1):
            print(f"{idx}. {item}")

        item_choice = int(input(f"\nSelect the number corresponding to the {forecast_by} you want to forecast: ")) - 1
        if item_choice < 0 or item_choice >= len(unique_items):
            print("Invalid selection.")
            return

        selected_item = unique_items[item_choice]
        print(f"\nYou selected: {selected_item}")

        steps = int(input("Enter the number of periods (days/weeks/months) to forecast (e.g., 30, 60, 90): "))
        freq = input("Enter the frequency of the forecast ('D' for days, 'W' for weeks, 'M' for months): ").strip().upper()
        group_data = aggregated_data[aggregated_data[forecast_by] == selected_item]

        if group_data.empty:
            print(f"No data found for {selected_item}.")
            return

        print(f"\nForecasting for {forecast_by}: {selected_item}")

        most_recent_date = group_data['InvoiceDate'].max()
        forecast_index = pd.date_range(start=most_recent_date + pd.Timedelta(days=1), periods=steps, freq=freq)
        best_model, final_forecast, metrics = forecasting_pipeline(group_data, steps=steps)

        if best_model:
            print(f"Best Model: {best_model}")
            print("Metrics:", metrics)

            forecast_df = pd.DataFrame({'Forecasted Revenue': final_forecast}, index=forecast_index)
            existing_data = group_data[['InvoiceDate', 'RevenueAmount']].rename(columns={'RevenueAmount': 'Actual Revenue'})
            combined_df = pd.concat([existing_data.set_index('InvoiceDate'), forecast_df])

            metadata_string = (
                f"Selected Item: {selected_item} | "
                f"Customer Filter: {selected_customer} | "
                f"SalesPerson Filter: {selected_salesperson} | "
                f"Location Filter: {selected_location}"
            )

            metadata_df = pd.DataFrame({0: [metadata_string]})
            combined_df_with_metadata = pd.concat([metadata_df, combined_df])

            download_excel = input("Would you like to download the forecast data as an Excel file? (yes/no): ").strip().lower()
            if download_excel == 'yes':
                excel_filename = f"{selected_item}_forecast.xlsx"
                with pd.ExcelWriter(excel_filename, engine='openpyxl') as writer:
                    metadata_df.to_excel(writer, index=False, header=False, startrow=0, startcol=0)
                    combined_df.to_excel(writer, startrow=1)
                print(f"Data saved to {excel_filename}.")
            else:
                print("Excel download skipped.")
                
        try:
            print("Plotting forecast results...")
            print("Actual dates range:", group_data['InvoiceDate'].min(), "-", group_data['InvoiceDate'].max())
            print("Forecast dates range:", forecast_index.min(), "-", forecast_index.max())
            print("Actual data length:", len(group_data['RevenueAmount']))
            print("Forecast data length:", len(final_forecast))

            plt.figure(figsize=(12, 6))
            plt.plot(group_data['InvoiceDate'], group_data['RevenueAmount'], label='Actual Revenue', color='blue')
            plt.plot(forecast_index, final_forecast, label='Forecasted Revenue', color='orange')
            plt.title(f"Forecasting Revenue for {selected_item}")
            plt.xlabel('Date')
            plt.ylabel('Revenue Amount')
            plt.xticks(rotation=45)
            plt.legend()
            plt.tight_layout()
            plt.show()

        except Exception as e:
            print(f"Error while plotting: {e}")

        if best_model:
            print(f"Best Model: {best_model}")
        else:
            print("No valid model was generated.")


In [ ]:
if __name__ == "__main__":
    main(df1_item)

Would you like to generate the 'full' 18-month forecast or a 'single' forecast? (Enter 'full' or 'single'):  full


Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into trai

/var/folders/8l/psd7bpq137j_d5v8d7ntz0200000gn/T/ipykernel_27318/2411138453.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group_df.ffill(inplace=True)  # Fill missing values with forward fill


Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.


/var/folders/8l/psd7bpq137j_d5v8d7ntz0200000gn/T/ipykernel_27318/2411138453.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group_df.ffill(inplace=True)  # Fill missing values with forward fill


...

Not enough data to split into train and test.
Not enough data to split into train and test.


/var/folders/8l/psd7bpq137j_d5v8d7ntz0200000gn/T/ipykernel_27318/2411138453.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group_df.ffill(inplace=True)  # Fill missing values with forward fill
/var/folders/8l/psd7bpq137j_d5v8d7ntz0200000gn/T/ipykernel_27318/2411138453.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group_df.ffill(inplace=True)  # Fill missing values with forward fill


Not enough data to split into train and test.
Insufficient data to forecast for Part Number PINNACLE-4-F|7 at Location 080|7. Skipping.
Not enough data to split into train and test.
Insufficient data to forecast for Part Number IRONWOOD-010-V|7 at Location 020|7. Skipping.
Not enough data to split into train and test.


/var/folders/8l/psd7bpq137j_d5v8d7ntz0200000gn/T/ipykernel_27318/2411138453.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group_df.ffill(inplace=True)  # Fill missing values with forward fill
/var/folders/8l/psd7bpq137j_d5v8d7ntz0200000gn/T/ipykernel_27318/2411138453.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  group_df.ffill(inplace=True)  # Fill missing values with forward fill
/var/folders/8l/psd7bpq137j_d5v8d7ntz0200000gn/T/ipykernel_27318/2411138453.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is de

Not enough data to split into train and test.
Not enough data to split into train and test.
Not enough data to split into train and test.
